In [1]:
import json
import pprint
import csv
import os

In [4]:
rootdir = '/Users/rdubnic2/Desktop/KD code and data/restricted_meta' # UPDATE THIS FOR EACH DIRECTORY

path_list = []

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        filepath = os.path.join(subdir, file)
        path_list.append(filepath)
# path_list.remove('/home/dcuser/Desktop/htrc-metadata/meta-chunks/pd_open_access/.~lock.open_segment_aa#')
# ABOVE LINE CAN OPTIONALLY BE USED TO REMOVE ANY HIDDEN FILES THAT ARE LIKELY TO BE FOUND IN THE DIECTORY ON LINUX/MAC

path_list

['/Users/rdubnic2/Desktop/KD code and data/restricted_meta/restricted_segment_ab.jsonl',
 '/Users/rdubnic2/Desktop/KD code and data/restricted_meta/restricted_segment_aa.jsonl',
 '/Users/rdubnic2/Desktop/KD code and data/restricted_meta/restricted_segment_ac.jsonl']

In [5]:
data_2 = []

for path in path_list:
    with open(path, 'r') as fin: # file path to directory--WILL NEED TO CHANGE THIS EACH RUN
        for line in fin:
            row = json.loads(line)
            data_2.append(row)

In [6]:
data_2

[{'leader': '01459nam a22002893u 4500',
  'fields': [{'001': '011371008'},
   {'003': 'MiAaHDL'},
   {'005': '20120302000000.0'},
   {'006': 'm        d        '},
   {'007': 'cr bn ---auaua'},
   {'007': 'kh bc    '},
   {'008': '^^^^^^||||||||||||nnn^^^^^^^f000^kn|||^d'},
   {'024': {'subfields': [{'a': 'c12 Pl7'}], 'ind1': '8', 'ind2': ' '}},
   {'035': {'subfields': [{'a': 'sdr-mdl.umn22321'}],
     'ind1': ' ',
     'ind2': ' '}},
   {'042': {'subfields': [{'a': 'dc'}], 'ind1': ' ', 'ind2': ' '}},
   {'245': {'subfields': [{'a': 'Geologic atlas of Goodhue County, Minnesota, C-12, Part B, Plate 7, Bedrock and  Water-Table Hydrogeology'}],
     'ind1': '0',
     'ind2': '0'}},
   {'260': {'subfields': [{'b': 'Minnesota Geological Survey, 2642 University Avenue, St. Paul Minnesota 55114'},
      {'c': '2003'}],
     'ind1': ' ',
     'ind2': ' '}},
   {'500': {'subfields': [{'a': 'United States'}], 'ind1': ' ', 'ind2': ' '}},
   {'500': {'subfields': [{'a': 'Minnesota'}], 'ind1': ' '

In [16]:
# data = []
# with open('/home/dcuser/Desktop/htrc-metadata/meta-chunks/pd_open_access/open_segment_dc', 'r') as fin: # file path to directory--WILL NEED TO CHANGE THIS EACH RUN
#     for line in fin:
#         row = json.loads(line)
#         data.append(row)

In [7]:
fielddata = {}

for entry in data_2:
    fields = entry['fields']
    for pair in fields:
        fieldnum = list(pair.keys())[0]
        
        if fieldnum not in fielddata:
            fielddata[fieldnum] = {'count': 1, 'subfields': {}}
        else:
            fielddata[fieldnum]['count'] += 1
        payload = list(pair.values())[0]
        if isinstance(payload, dict):
            subfields = payload['subfields']
            for s in subfields:
                subcode, subvalue = list(s.items())[0]
                if subcode not in fielddata[fieldnum]['subfields']:
                    fielddata[fieldnum]['subfields'][subcode] = 1
                else:
                    fielddata[fieldnum]['subfields'][subcode] += 1
        else:
            subfields = "missing"
        

In [8]:
## Creating the fields records for the csv

allfieldsseen = list(fielddata.keys())
allsubfieldsdseen = []

for field, values in fielddata.items():
    subs = list(values['subfields'].keys())
    for s in subs:
        if s not in allsubfieldsdseen:
            allsubfieldsdseen.append(s)
            
allfieldsseen.sort()
allsubfieldsdseen.sort()
        

In [9]:
# now let's go through the original data rows and try to smash this into the full csv possible

rows = []

for chunk in data_2:
    row = []
    payload = chunk['fields']
    flat = {}
    for item in payload:
        field, value = list(item.items())[0] # get the tuple of the field and unpack
        flat[field] = value
    for f in allfieldsseen:
        if f in flat:
            if isinstance(flat[f], dict):
                c = flat[f]['subfields']
                row.append(c)
            else:
                row.append(flat[f])
        else:
            row.append('MISSING')
    rows.append(row)

In [15]:
i = 1

for path in path_list:
    i_str = str(i)
    with open(path[:-6]+i_str+'.csv', 'w') as out: # generates CSVs from the file names of jsonl, with an iterative number at the end
        csvout = csv.writer(out)
        csvout.writerow(allfieldsseen)
        csvout.writerows(rows)
        i += 1

In [17]:
# data[0]

In [18]:
# print("Complete")